In [1]:
# importando bibliotecas
import gurobipy as gp
from gurobipy import GRB

# Uncapacitated facility location problem

Considere a formulação do problema de localização de facilidades não capacitado.

$
\begin{align*}
\min \ & \sum_{i \in M} \sum_{j \in N} c_{ij} x_{ij} + \sum_{j \in N} f_j y_j \\
& \sum_{j \in N} x_{ij} = 1 \ \mbox{ for } i=1, \ldots, m \\
& \sum_{i \in M} x_{ij} \leq m \cdot y_j \mbox{ for } j \in N \\
& x_{ij} \geq 0 \mbox{ for } i \in M, j \in N \\
& y_j \in \{ 0, 1 \} \mbox{ for } j \in N
\end{align*}
$

Resolva uma instância do uncapacitated facility problem onde $f_j$ é o custo do depósito $j$ aberto, e $c_{ij}$ é o custo da satisfação da demanda de todos os clientes $i$'s pelo depósito $j$, com 

* número de clientes: 6

* número de depósitos: 5

* custos fixo:

$
f = [4,3,4,4,7]
$

* custo de transporte:

$
c = 
\left[
\begin{array}{ccccc}
12 & 13 & 6 & 0 & 1 \\
8 & 4 & 9 & 1 & 2 \\
2 & 6 & 6 & 0 & 1 \\
3 & 5 & 2 & 1 & 8 \\
8 & 0 & 5 & 10 & 8 \\
2 & 0 & 3 & 4 & 1
\end{array}
\right]
$

## Dados

In [2]:
# dimensões
m = 6
n = 5

# custos fixo
f = [4, 3, 4, 4, 7]

# custo de transporte
c = [[12, 13, 6, 0, 1],
     [8, 4, 9, 1, 2],
     [2, 6, 6, 0, 1],
     [3, 5, 2, 1, 8],
     [8, 0, 5, 10, 8],
     [2, 0, 3, 4, 1]]

# conjuntos
N = range(n)
M = range(m)

## Formulação MIP

In [3]:
# criando o modelo
model = gp.Model("ufl")

# definindo a variável binária y
y = model.addVars(n,vtype=GRB.BINARY,obj=f,name="y")

# definindo a variável de produção x
x = model.addVars(m,n,obj=c,name="x")

# restrição de demanda
model.addConstrs((x.sum(i) == 1 for i in M), "demanda")

# restrição link entre y e x
model.addConstrs((x.sum('*', j) <= m*y[j] for j in N ), "limite")

# problema de minimização
model.modelSense = GRB.MINIMIZE

# escrevendo o modelo
model.write('facility.lp')

Academic license - for non-commercial use only - expires 2023-01-08
Using license file /opt/gurobi912/gurobi.lic


## Definindo os parametros

In [4]:
# configurando parametros
model.Params.TimeLimit = 60 # define tempo limite
model.Params.method = 1
model.Params.Threads = 1

Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0


## Resolvendo o problema

In [5]:
# resolvendo o problema
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads
Optimize a model with 11 rows, 35 columns and 65 nonzeros
Model fingerprint: 0x8463997b
Variable types: 30 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros
Variable types: 30 continuous, 5 integer (5 binary)

Root relaxation: objective 5.666667e+00, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.66667    0    2          -    5.66667      -     -    0s
H    0     0                       9.0000000    5.66667  37.0%     -    0s
     0     0     cutoff    0         9.00000    9.00000  0.00%     -    0s

Exp

## Imprimindo Solução MIP

In [6]:
if model.status == GRB.OPTIMAL:
    print('Valor ótimo: %g' % model.objVal)
    for j in model.getVars():
        if j.x > 0:
            print('%s = %g' % (j.varName, j.x))
else:
    print('Otimização finalizou com status %d' % (model.status))
    
print('Tempo: ', round(model.Runtime,2))

Valor ótimo: 9
y[1] = 1
y[3] = 1
x[0,3] = 1
x[1,3] = 1
x[2,3] = 1
x[3,3] = 1
x[4,1] = 1
x[5,1] = 1
Tempo:  0.03


In [7]:
# verificando se o modelo é MIP
if model.isMIP == 1:
    print('O modelo é um MIP')
else:
    print('O modelo não é um MIP')

O modelo é um MIP


In [8]:
# relaxando a variáveis inteiras
for v in model.getVars():
    if v.vType != GRB.CONTINUOUS:
        v.vType = GRB.CONTINUOUS

In [9]:
# resolvendo o problema relaxado
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads
Optimize a model with 11 rows, 35 columns and 65 nonzeros
Model fingerprint: 0x487a44f7
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+00   7.500000e-01   0.000000e+00      0s
       2    5.6666667e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  5.666666667e+00


## Imprimindo solução da relaxação linear

In [10]:
# imprimindo solução
if model.status == GRB.OPTIMAL:
    print('Valor ótimo: %g' % model.objVal)
    for j in model.getVars():
        if j.x > 0:
            print('%s = %g' % (j.varName, j.x))
else:
    print('Otimização finalizou com status %d' % (model.status))
    
print('tempo: ', round(model.Runtime,2))

Valor ótimo: 5.66667
y[1] = 0.333333
y[3] = 0.666667
x[0,3] = 1
x[1,3] = 1
x[2,3] = 1
x[3,3] = 1
x[4,1] = 1
x[5,1] = 1
tempo:  0.01


## Tarefa


1. Implemente uma heurística gulosa para encontrar uma solução viável para o problema. Sugestão: Para servir cada cliente utilize o depósito que serve o respectivo cliente com o menor custo.

2. Implemente uma estratégia de busca local vista em sala de aula: Encontrar uma vizinhança próximo da solução viável inicial e em seguida buscar uma nova solução nesta vizinhança. Se a nova solução for melhor que a solução anterior, a nova solução passa a ser a solução viável para o problema e o procedimento se repete. Caso contrário a solução viável atual é um ótimo local com respeito a vizinhança atual, e a heurı́stica termina.